In [17]:
import json
import os
import time
import ipyparallel as ipp

import matplotlib.pyplot as plt
import torch
import dfca as DFCA
import ifca as IFCA

import numpy as np

from util import *

Reads Config file and prepares the arguments you can choose in the config_dfca.json/config_ifca.json

In [18]:
rc = ipp.Cluster(n=3).start_and_connect_sync()
rc.activate()

rc.ids 

dview = rc[:]

Starting 3 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>
100%|██████████| 3/3 [00:06<00:00,  2.06s/engine]


In [19]:
%%px --targets all

import sys
import os
sys.path.append(os.getcwd())

seeds = [11,12]
learning_rates = [0.001, 0.01]

dfca_li = []
dfca_gi = []
ifca = []

In [20]:
seeds = [11,12]
learning_rates = [0.001, 0.01]

dfca_li = []
dfca_gi = []
ifca = []

Running the dfca-gi experiment

In [ ]:
%%px --target 0 --noblock
import dfca as DFCA
import json
import time
import torch

with open("config_dfca_gi.json", "r") as read_file:
    config = json.load(read_file)

for lr in learning_rates:
    config['lr'] = lr
    res_final = []

    for seed in seeds:
        print(f"---------------------------------DFCA-LI, lr: {lr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = DFCA.TrainMNISTCluster(config, device)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    dfca_gi.append(res_mean)



<AsyncResult(%px): pending>

Running the dfca-li experiment


In [ ]:
%%px --target 1 --noblock
import dfca as DFCA
import json
import time
import torch

with open("config_dfca_li.json", "r") as read_file:
    config = json.load(read_file)

for lr in learning_rates:
    config['lr'] = lr
    res_final = []

    for seed in seeds:
        print(f"---------------------------------DFCA-LI, lr: {lr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = DFCA.TrainMNISTCluster(config, device)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    dfca_li.append(res_mean)



<AsyncResult(%px): pending>

Running the ifca experiment

In [ ]:
%%px --target 2 --noblock
import ifca as IFCA
import json
import time
import torch

with open("config_ifca.json", "r") as read_file:
    config = json.load(read_file)

for lr in learning_rates:
    config['lr'] = lr
    res_final = []

    for seed in seeds:
        print(f"---------------------------------IFCA, lr: {lr}, seed: {seed}---------------------------------")
        start_time = time.time()
        config['data_seed'] = seed
        config['train_seed'] = config['data_seed']

        print("config:",config)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {device}")

        exp = IFCA.TrainMNISTCluster(config, device)
        exp.setup()
        res = exp.run()
        res_final.append([[r['test']['loss'] for r in res], [r['test']['acc'] for r in res], [r['test']['cl_acc'] for r in res]])
        del exp
        duration = (time.time() - start_time)
        print("---train cluster Ended in %0.2f hour (%.3f sec) " % (duration/float(3600), duration))

    res_mean = np.mean(res_final, axis=0).tolist()
    ifca.append(res_mean)



<AsyncResult(%px): pending>

Experiment Final Results: 

In [24]:
rc.wait_interactive()
%pxresult
dfca_gi = rc[0].pull('dfca_gi', block=True)
dfca_li = rc[1].pull('dfca_li', block=True)
ifca = rc[2].pull('ifca', block=True)

os.makedirs('graphs', exist_ok=True)

plt.figure(figsize=(10,5))
plt.plot([r['test']['loss'] for r in ifca], label='ifca', color='blue')
plt.plot([r['test']['loss'] for r in dfca_li], label='dfca-li', color='red')
plt.plot([r['test']['loss'] for r in dfca_gi], label='dfca-gi', color='green')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Test Loss per Epoch')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join('graphs', 'train_loss.png'))

plt.figure(figsize=(10,5))
plt.plot([r['test']['acc'] for r in ifca], label='ifca', color='blue')
plt.plot([r['test']['acc'] for r in dfca_li], label='dfca-li', color='red')
plt.plot([r['test']['acc'] for r in dfca_gi], label='dfca-gi', color='green')
plt.xlabel('epoch')
plt.ylabel('test accuracy')
plt.title('Test Accuracy per Epoch')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join('graphs', 'test_acc.png'))

plt.figure(figsize=(10,5))
plt.plot([r['test']['cl_acc'] for r in ifca[:3]], label='ifca', color='blue')
plt.plot([r['test']['cl_acc'] for r in dfca_li[:3]], label='dfca-li', color='red')
plt.plot([r['test']['cl_acc'] for r in dfca_gi[:3]], label='dfca-gi', color='green')
plt.xlabel('epoch')
plt.ylabel('cluster acc')
plt.title('Cluster Accuracy per Epoch')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join('graphs', 'cluster_acc.png'))

unknown: 100%|██████████| 3/3 [02:53<00:00, 57.69s/tasks]


RemoteError: [2:execute] TypeError: list indices must be integers or slices, not str